In [14]:
import torch
import torch.nn as nn
from MobileNetV2 import mobilenet_v2
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import numpy as np

In [15]:
save_path = 'save_model/ecg_id.pt'
model = torch.load(save_path)

In [16]:
# 超参数
input_size = 224
batch_size = 32
n_worker = 0
epochs = 150

In [17]:
test_path = "image/test_image"
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
data_transform = transforms.Compose([
        transforms.RandomResizedCrop(input_size, scale=(0.2, 1.0)), 
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
])
test_dataset = datasets.ImageFolder(test_path, transform=data_transform)
test_loader = DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, 
    num_workers=n_worker)

In [18]:
def test():       
    model.eval()
    gt_labels = []
    pred_labels = []
    with torch.no_grad():
        for data, label in test_loader:
            data, label = data.cuda(), label.cuda()
            output = model(data)
            preds = torch.argmax(output, 1)
            gt_labels.append(label.cpu().data.numpy())
            pred_labels.append(preds.cpu().data.numpy())
    gt_labels, pred_labels = np.concatenate(gt_labels), np.concatenate(pred_labels)
    acc = np.sum(gt_labels==pred_labels)/len(pred_labels)
    print('Accuracy: {:6f}'.format(acc))

In [19]:
test()

Accuracy: 0.573889
